## Notebook for visualizing and sorting through eigenmode's purity scores

The goal is to extract canonical functional networks from Yeo et al.'s 2017 paper and compare each canonical network with our own complex Laplacian eigenmodes. 

Some methods overview:
From Basset/Betzel text: similarity between "modules" (in our case, eigen modes) and canonical cognitive systems is assessed with an overlap score: 

Let $x = [x_1,...,x_n]$ be a binary module vector whose element $x_i$ is equal to 1 if region $i$ is assigned to that module and $0$ otherwise. Similarly, let $y = [y_1,...,y_n]$ be a binary system vector whose element $y_i$ is equal to 1 if region $i$ is assigned to that system and $0$ otherwise. The overlap of $x$ and $y$ is given by $s(x,y) = \frac{x \cap y}{x \cup y}$ and the standardized version of this measure is:

$z(x,y) = \frac{s(x,y) - \mu}{\sigma}$

Where $\mu$ and $\sigma$ are the mean and standard deviation of $s(x,y)$ estimated by randomly permuting the elements of $x$ and $y$ (1000 repetitions). Large and positive values of $z(x,y)$ therefore indicate greater overlap than expected by chance, given the module's and system's sizes. For convenience, each module was assigned to the system with which it exhibited the greatest standardized overlap. 

Each "module" (eigen mode) was associated with seven overlap scores - one for each of the seven systems: $ S = [s_1, ... , s_7]$, where $s_i$ is the overlap of the "module"(eigen mode) with the $i$th system. The purity of that module quantifies the extent to which it overlaps with many systems (low purity) or few systems (high purity). "Module" (eigen mode) purity is defined by first normalizing $S$ so that its elements sum to 1. Then we compute an entropy over these elements as _purity_ $= -\sum{_i}{s'_i \log_2 s'_i}$, where $s'_i$ is the normalized overlap of a module with the $i$th system.


In [ ]:
%%capture --no-display

import os
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#spectrome imports:
from brain import Brain
from utils import functions, path
from forward import e